In [1]:
import h5py
import numpy as np

from e2c_create import create_e2c_var_wl

from keras import backend as K
from keras.layers import Input
from keras.models import Model

import matplotlib.pyplot as plt
plt.set_cmap('jet')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# GPU memory management
import tensorflow as tf

Using TensorFlow backend.


In [2]:
# tf.session specification
# TensorFlow wizardry
config = tf.ConfigProto()

# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total of half the GPU memory to be allocated
config.gpu_options.per_process_gpu_memory_fraction = 0.5

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

In [3]:
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_BHP_RATE/'
# data_dir = '/data/cees/zjin/lstm_rom/datasets/9W_MS_BHP_RATE/'
data_dir = '/data3/Astro/personal/zjin/datasets/9W_MS_BHP_RATE_GAU/'
# data_dir = '/data3/Astro/personal/zjin/datasets/7W_CHA/'

output_dir = '/data3/Astro/lstm_rom/e2c_larry/saved_models/' # load model data

case_name = '9w_ms_bhp_rate'
# case_name = '7w_cha'

# target_suffix = '_fix_wl_rel_2'
# target_suffix = '_fix_wl_rel_8' # the dataset being evaluated here
target_suffix = '_var_wl_rel_1'

# case_suffix = '_fix_wl_rel_2'
# case_suffix = '_fix_wl_rel_8'
case_suffix = '_var_wl_rel_1'

train_suffix = '_with_p'

model_suffix = '_flux_loss'
# model_suffix = '_no_fl'
# model_suffix = '_ae_no_l2_ep_10' # only in 7w_cha

num_t = 20
run_eval = 100
run_train = 300
num_eval = num_t * run_eval
num_train = num_t * run_train

dt = 100

eval_file = case_name + '_e2c_eval' + target_suffix + train_suffix + '_n%d_dt%dday_nt%d_nrun%d.mat'%(num_eval, dt, num_t, run_eval)
train_file = case_name + '_e2c_train' + target_suffix + train_suffix + '_n%d_dt%dday_nt%d_nrun%d.mat'%(num_train, dt, num_t, run_train)

state_file = case_name + '_train_n_400_full'
ctrl_file = case_name + '_norm_bhps_n_400'

state_data = state_file + target_suffix + '.mat'
ctrl_data = ctrl_file + target_suffix + '.mat'

## Load E2C model

In [4]:
latent_dim, u_dim = 50, 50
input_shape = (60, 60, 2) # change from _with_p to _no_p
encoder, decoder, transition, wc_encoder = create_e2c_var_wl(latent_dim, u_dim, input_shape)

latent_dim, learning_rate, epoch = 50, 1e-4, 20

In [5]:
encoder.load_weights(output_dir + 'e2c_encoder_dt_'+case_name+case_suffix+train_suffix+model_suffix+'_nt%d_l%d_lr%.0e_ep%d.h5' % (num_train, latent_dim, learning_rate, epoch))
decoder.load_weights(output_dir + 'e2c_decoder_dt_'+case_name+case_suffix+train_suffix+model_suffix+'_nt%d_l%d_lr%.0e_ep%d.h5' % (num_train, latent_dim, learning_rate, epoch))
transition.load_weights(output_dir + 'e2c_transition_dt_'+case_name+case_suffix+train_suffix+model_suffix+'_nt%d_l%d_lr%.0e_ep%d.h5' % (num_train, latent_dim, learning_rate, epoch))
wc_encoder.load_weights(output_dir + 'e2c_wc_encoder_dt_'+case_name+case_suffix+train_suffix+model_suffix+'_nt%d_l%d_lr%.0e_ep%d.h5' % (num_train, latent_dim, learning_rate, epoch))

## One step prediction

In [6]:
hf_r = h5py.File(data_dir + eval_file, 'r')
state_t_eval = np.array(hf_r.get('state_t'))
state_t1_eval = np.array(hf_r.get('state_t1'))
bhp_eval = np.array(hf_r.get('bhp'))
dt_eval = np.array(hf_r.get('dt'))
wl_mask_eval = np.array(hf_r.get('wl_mask'))
hf_r.close()

In [7]:
# num_eval = 20 # pick 20 out of 2000 evals
# state_t_eval = state_t_eval[:num_eval, ...]
# state_t1_eval = state_t1_eval[:num_eval, ...]
# bhp_eval = bhp_eval[:num_eval, ...]
# wl_mask_eval = wl_mask_eval[:num_eval, ...]

In [8]:
print(state_t_eval.shape)
print(state_t1_eval.shape)
print(bhp_eval[:,:5].min().min())
print(bhp_eval[:,:5].max().max())
print(bhp_eval.shape)

(2000, 60, 60, 2)
(2000, 60, 60, 2)
-0.14284373895778216
0.9101197994449696
(2000, 60, 60, 2)


In [9]:
hf_r = h5py.File(data_dir + train_file, 'r')
state_t_train = np.array(hf_r.get('state_t'))
state_t1_train = np.array(hf_r.get('state_t1'))
bhp_train = np.array(hf_r.get('bhp'))
dt_train = np.array(hf_r.get('dt'))
wl_mask_train = np.array(hf_r.get('wl_mask'))
hf_r.close()

In [10]:
print(state_t_train.shape)
print(state_t1_train.shape)
print(bhp_train[:,:5].min().min())
print(bhp_train[:,:5].max().max())
print(bhp_train.shape)

(6000, 60, 60, 2)
(6000, 60, 60, 2)
-0.1428273875727304
0.9555559114598056
(6000, 60, 60, 2)


In [11]:
xt = Input(shape=input_shape)
xt1 = Input(shape=input_shape)
ut = Input(shape=input_shape)
dt = Input(shape=(1,))
wl_mask = Input(shape=(60, 60)) # both prod and inj

In [12]:
zt = encoder(xt)
xt_rec = decoder(zt)

ut_encoded = wc_encoder(ut)

zt1_pred = transition([zt, ut_encoded, dt])
xt1_pred = decoder(zt1_pred)

e2c_model = Model([xt, ut, dt], [xt_rec, xt1_pred])

In [ ]:
[state_t_rec_eval, state_t1_pred_eval] = e2c_model.predict([state_t_eval, bhp_eval, dt_eval])

In [ ]:
[state_t_rec_train, state_t1_pred_train] = e2c_model.predict([state_t_train, bhp_train, dt_train])

In [ ]:
print(state_t_rec_eval.shape)
print(state_t1_pred_eval.shape)

print(state_t_rec_train.shape)
print(state_t1_pred_train.shape)

In [ ]:
for k in range(10):
    plt.figure(figsize=(20,10))
    plt.subplot(1, 3, 1)
    plt.imshow(state_t_eval[k, :, :, 0])
    plt.clim([0.1, 0.7])
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('True Sat t')
    
    plt.subplot(1, 3, 2)
    plt.imshow(state_t_rec_eval[k, :, :, 0])
    plt.clim([0.1, 0.7])
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('Reconstructed Sat t')
    
    plt.subplot(1, 3, 3)
    plt.imshow(np.fabs(state_t_rec_eval[k, :, :, 0] - state_t_eval[k, :, :, 0]))
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('Reconstruction error t')
plt.show()

In [ ]:
for k in range(5):
    plt.figure(figsize=(20,10))
    plt.subplot(1, 3, 1)
    plt.imshow(state_t_train[k, :, :, 0])
    plt.clim([0.1, 0.7])
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('True Sat t')
    
    plt.subplot(1, 3, 2)
    plt.imshow(state_t_rec_train[k, :, :, 0])
    plt.clim([0.1, 0.7])
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('Reconstructed Sat t')
    
    plt.subplot(1, 3, 3)
    plt.imshow(np.fabs(state_t_rec_train[k, :, :, 0] - state_t_train[k, :, :, 0]))
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('Reconstruction error t')
    
plt.show()

In [ ]:
p_max = 425
p_min = 250
p_diff = p_max - p_min

state_t_eval_plot = state_t_eval * p_diff + p_min
state_t_rec_plot = state_t_rec * p_diff + p_min
state_t1_eval_plot = state_t1_eval * p_diff + p_min
state_t1_pred_plot = state_t1_pred * p_diff + p_min

In [ ]:
for k in range(num_eval):
    plt.figure(figsize=(20,10))
    plt.subplot(2, 3, 1)
    plt.imshow(state_t_eval_plot[k, :, :, 1])
#     plt.clim([4150, 4600])
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('True Pres t')
    
    plt.subplot(2, 3, 2)
    plt.imshow(state_t_rec_plot[k, :, :, 1])
#     plt.clim([4150, 4600])
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('Reconstructed Pres t')
    
    plt.subplot(2, 3, 3)
    plt.imshow(np.fabs(state_t_rec_plot[k, :, :, 1] - state_t_eval_plot[k, :, :, 1]))
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('Reconstruction error t')
    
    plt.subplot(2, 3, 4)
    plt.imshow(state_t1_eval_plot[k, :, :, 1])
#     plt.clim([4150, 4600])
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('True Pres t+1')
    
    plt.subplot(2, 3, 5)
    plt.imshow(state_t1_pred_plot[k, :, :, 1])
#     plt.clim([4150, 4600])
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('Predicted Pres t+1')
    
    plt.subplot(2, 3, 6)
    plt.imshow(np.fabs(state_t1_pred_plot[k, :, :, 1] - state_t1_eval_plot[k, :, :, 1]))
    plt.colorbar(fraction=0.046)
    plt.xticks([])
    plt.yticks([])
    plt.title('Prediction Error t+1')
plt.show()

## Sequential prediction

In [ ]:
hf_r = h5py.File(data_dir + state_data, 'r')
# sat = np.array(hf_r.get('sat'))
# pres = np.array(hf_r.get('pres'))
sat_t = np.array(hf_r.get('sat_t')).reshape(400, 20, 60, 60)
pres_t = np.array(hf_r.get('pres_t')).reshape(400, 20, 60, 60)
hf_r.close()

In [ ]:
hf_r = h5py.File(data_dir + ctrl_data, 'r')
# bhp0 = np.array(hf_r.get('bhp'))
# rate0 = np.array(hf_r.get('rate'))
wc_encoded_t = np.array(hf_r.get('wc_encoded_t'))
hf_r.close()
# bhp -= 0.45

In [ ]:
print(wc_encoded_t.shape)
print(sat_t.shape)
print(pres_t.shape)

In [ ]:
# bhp = np.concatenate((bhp0,rate0),axis=1)
# print(bhp.shape)

In [ ]:
# sat = sat.T.reshape((400, 201, 3600))
# pres = pres.T.reshape((400, 201, 3600))

In [ ]:
test_case0 = np.zeros((25,4))
a = np.array(range(75,400,100))[np.newaxis,:]
b = np.array(range(25))[:,np.newaxis]
print(a.shape)
print(b.shape)
test_case = (test_case0 + a + b).T.reshape(100)
test_case = np.array(test_case).astype(int)

# test_case = np.array(test_case)
print(test_case.shape)

In [ ]:
# ind_case = np.array(range(0, 100, 25)) + 10

ind_case = np.array([10, 25, 77, 97])
print(ind_case)

In [ ]:
num_case = test_case.shape[0] # 4
num_tstep = 20
sat_pred = np.zeros((num_case, num_tstep, 60, 60, 1))
pres_pred = np.zeros((num_case, num_tstep, 60, 60, 1))

In [ ]:
# t_steps = list(np.arange(0,100,10))
t_steps = np.arange(0,200,200//num_tstep)
print(t_steps)
# num_tstep = len(t_steps)

In [ ]:
dt = 10
t_steps1 = (t_steps + dt).astype(int)
print(t_steps1)

In [ ]:
indt_del = t_steps1 - t_steps
indt_del = indt_del / max(indt_del)
print(indt_del.shape)

In [ ]:
# tmp = np.array(range(num_tstep)) - 1
# tmp1 = np.array(range(num_tstep))
# tmp[0] = 0

# print(tmp)
# print(tmp1)
# print(bhp.shape)

In [ ]:
num_prod = 5
num_inj = 4
num_well = num_prod + num_inj

num_all_case = 400
num_ctrl = 20


In [ ]:
print(num_ctrl)
print(num_tstep)

In [ ]:
# bhp_b0 = bhp.reshape(num_all_case, num_well, num_ctrl)
# bhp_b1 = np.repeat(bhp_b0[..., np.newaxis], num_tstep // num_ctrl, axis=3)
# assert num_tstep // num_ctrl * num_ctrl == num_tstep, "no exaxt division num_step = %d, num_ctrl=%d"%(num_tstep, num_ctrl)

# print(bhp_b1.shape)
# bhp_b2 = bhp_b1.reshape(num_all_case, num_well, num_tstep)
# print(bhp_b2.shape)

In [ ]:
# bhp_tt = bhp_b2[:,:, tmp]
# bhp_tt1 = bhp_b2[:,:, tmp1]
# print(bhp_tt.shape)
# print(bhp_tt1.shape)

# bhp_tt0 = np.concatenate((bhp_tt, bhp_tt1), axis=1)
# bhp_t = np.swapaxes(bhp_tt0,1,2)
# print(bhp_t.shape)
# print(bhp[0,:20])
# print(bhp_t[0,:,9])

In [ ]:
# bhp_seq = bhp_t[test_case, :, :]
# print(bhp_seq.shape)

In [ ]:
bhp_seq = wc_encoded_t[test_case, ...]
print(bhp_seq.shape)

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(141)
plt.imshow(bhp_seq[ind_case[0],1,:,:,0])
plt.colorbar(fraction=0.046)
plt.subplot(142)
plt.imshow(bhp_seq[ind_case[1],1,:,:,0])
plt.colorbar(fraction=0.046)
plt.subplot(143)
plt.imshow(bhp_seq[ind_case[2],1,:,:,0])
plt.colorbar(fraction=0.046)
plt.subplot(144)
plt.imshow(bhp_seq[ind_case[3],1,:,:,0])
plt.colorbar(fraction=0.046)

In [ ]:
sat_t_seq = sat_t[test_case, 0, ...].reshape((num_case, 60, 60, 1)) # 4 here is the 4th timestep, t = 8
pres_t_seq = pres_t[test_case, 0, ...].reshape((num_case, 60, 60, 1))
print(sat_t_seq.shape)
print(pres_t_seq.shape)
print("num_case:{}".format(num_case))

In [ ]:
# plt.figure(figsize=(15,6))
# for k in range(len(ind_case)):
#     plt.subplot(2,4,k+1)
#     plt.imshow(sat_t_seq[ind_case[k], :, :, 0])
#     plt.colorbar(fraction=0.046)
    
#     plt.subplot(2,4,k+5)
#     plt.imshow(pres_t_seq[ind_case[k], :, :, 0])
#     plt.colorbar(fraction=0.046)
# plt.show()

In [ ]:
print(sat_t_seq.shape)
print(sat_pred.shape)
print(pres_t_seq.shape)
print(pres_pred.shape)
print(bhp_seq.shape)

In [ ]:
state_t_seq = np.concatenate((sat_t_seq, pres_t_seq),axis=3)
state_pred = np.concatenate((sat_pred, pres_pred),axis=4)
print(state_t_seq.shape)
print(state_pred.shape)

In [ ]:
# plt.figure(figsize=(15,3))
# for k in range(len(ind_case)):
#     plt.subplot(1,4,k+1)
#     plt.imshow(state_t_seq[ind_case[k], :, :, 0])
#     plt.colorbar(fraction=0.046)
# plt.show()

In [ ]:
for i_tstep in range(num_tstep):
    state_pred[:, i_tstep, ...] = state_t_seq.copy()
    dt_seq = np.ones((num_case,1)) * indt_del[i_tstep]
    [_, state_t1_seq] = e2c_model.predict([state_t_seq, bhp_seq[:,i_tstep,:], dt_seq])
    state_t_seq = state_t1_seq.copy()

In [ ]:
print(sat.shape)
print(pres.shape)
# print(state.shape)
print(state_pred.shape)

In [ ]:
# sat_seq_true = sat[test_case[ind_case], ...]
sat_seq_true = sat_t[test_case, ...]
# sat_seq_true = sat_seq_true[:, list(np.arange(0,200,10)), :]
print(sat_seq_true.shape)

# pres_seq_true = pres[test_case[ind_case], ...]
pres_seq_true = pres_t[test_case, ...]
# pres_seq_true = pres_seq_true[:, list(np.arange(0,200,10)), :]
print(pres_seq_true.shape)
state_seq_true = np.zeros((len(test_case),20,60,60,2))
state_seq_true[:,:,:,:,0] = sat_seq_true
state_seq_true[:,:,:,:,1] = pres_seq_true
print(state_seq_true.shape)

In [ ]:
sat_pred_plot = state_pred[:, :, :, :, 0] * s_diff + s_min
state_pred[:, :, :, :, 0] = state_pred[:, :, :, :, 0] * s_diff + s_min

In [ ]:
divide = 2
for k in range(4):
    print("Case num: %d"%ind_case[k])
    plt.figure(figsize=(16,5))
    for i_tstep in range(len(t_steps)//divide):
        plt.subplot(3, num_tstep//divide, i_tstep+1)
        plt.imshow(sat_pred_plot[ind_case[k], i_tstep*divide, :,:])
        plt.title('t=%d'%(t_steps[i_tstep*divide]*dt))
        plt.xticks([])
        plt.yticks([])
        plt.clim([0.1, 0.7])
        if i_tstep == 9:
            plt.colorbar(fraction=0.046) 
            
        
        plt.subplot(3, num_tstep//divide, i_tstep+1+num_tstep//divide)
        plt.imshow(state_seq_true[ind_case[k], i_tstep*divide, :, :,0].reshape((60,60)))
        plt.xticks([])
        plt.yticks([])
        plt.clim([0.1, 0.7])
        if i_tstep == 9:
            plt.colorbar(fraction=0.046)         
        
        plt.subplot(3, num_tstep//divide, i_tstep+1+2*num_tstep//divide)
        plt.imshow(np.fabs(state_seq_true[ind_case[k], i_tstep*divide, :,:, 0].reshape((60,60)) - sat_pred_plot[ind_case[k], i_tstep*divide, :,:]))
        plt.xticks([])
        plt.yticks([])
        plt.clim([0, 0.15])
        if i_tstep == 9:
            plt.colorbar(fraction=0.046) 

    plt.show()

In [ ]:
state_pred_plot = state_pred[:, :, :, :, 1] * p_diff + p_min
state_seq_true_plot = state_seq_true[:, :, :, :,1] * p_diff + p_min
print(state_pred_plot.shape)
print(state_seq_true_plot.shape)

In [ ]:
divide = 2
for k in range(4):
    print("Case num: %d"%ind_case[k])
    plt.figure(figsize=(16,5))
    for i_tstep in range(len(t_steps)//divide):
        plt.subplot(3, num_tstep//divide, i_tstep+1)
        plt.imshow(state_pred_plot[ind_case[k], i_tstep*divide, :, :])
        plt.title('t=%d'%(t_steps[i_tstep*divide]*dt))
        plt.xticks([])
        plt.yticks([])
#         plt.clim([4150, 4650])
        if i_tstep == 9:
            plt.colorbar(fraction=0.046) 
            
        
        plt.subplot(3, num_tstep//divide, i_tstep+1+num_tstep//divide)
        plt.imshow(state_seq_true_plot[ind_case[k], i_tstep*divide, :].reshape((60,60)))
        plt.xticks([])
        plt.yticks([])
#         plt.clim([4150, 4650])
        if i_tstep == 9:
            plt.colorbar(fraction=0.046)         
        
        plt.subplot(3, num_tstep//divide, i_tstep+1+2*num_tstep//divide)
        plt.imshow(np.fabs(state_seq_true_plot[ind_case[k], i_tstep*divide, :].reshape((60,60)) - state_pred_plot[ind_case[k], i_tstep*divide, :,:]))
        plt.xticks([])
        plt.yticks([])
#         plt.clim([0, 0.02])
        if i_tstep == 9:
            plt.colorbar(fraction=0.046) 

    plt.show()

In [ ]:
print(state_seq_true.shape)
print(state_pred.shape)
print(bhp_seq.shape)

In [ ]:
print(ind_case)

In [ ]:
train_case0 = np.zeros((75,4))
a = np.array(range(0,400,100))[np.newaxis,:]
b = np.array(range(75))[:,np.newaxis]
print(a.shape)
print(b.shape)
train_case = (train_case0 + a + b).T.reshape(300)
train_case = np.array(train_case)
print(train_case)

In [ ]:
print(sat.shape)
print(pres.shape)
sat_seq_train0 = sat[train_case.astype(int), :, :] 
pres_seq_train0 = pres[train_case.astype(int), :, :]
sat_seq_train = sat_seq_train0[:, np.arange(0,200,10), :]
pres_seq_train = pres_seq_train0[:, np.arange(0,200,10), :]
print(sat_seq_true.shape)
print(pres_seq_true.shape)

state_seq_train = np.zeros((300,20,3600,2))
state_seq_train[:,:,:,0] = sat_seq_train
state_seq_train[:,:,:,1] = pres_seq_train
print(state_seq_train.shape)

In [ ]:
bhp_train = bhp[train_case.astype(int),:]
bhp_eval = bhp[test_case.astype(int),:]
print(state_seq_true.shape)
print(state_pred.shape)
print(state_seq_train.shape)
print(bhp_seq.shape)
print(bhp_train.shape)
print(bhp_eval.shape)


In [ ]:
output_dir = '/data3/Astro/lstm_rom/e2c_larry/data/9w_ms_bhp_rate_fix_wl/'
# output_dir = '/data3/Astro/lstm_rom/e2c_larry/data/' + case_name + '/'

# old case may not have model_suffix in the result filename
# hf_w = h5py.File(output_dir + case_name + target_suffix + '_nt_%d_state_seq_pred_ctrl.mat'%(num_train), 'w')
hf_w = h5py.File(output_dir + case_name + target_suffix + model_suffix + '_nt_%d_state_seq_pred_ctrl.mat'%(num_train), 'w')
hf_w.create_dataset('true_seq', data=state_seq_true)
hf_w.create_dataset('pred_seq', data=state_pred)
hf_w.create_dataset('train_seq', data=state_seq_train)
hf_w.create_dataset('ctrl_seq', data=bhp_seq)
hf_w.create_dataset('train_ctrl', data=bhp_train)
hf_w.create_dataset('eval_ctrl', data=bhp_eval)
hf_w.close()

In [ ]:
print(num_train)